In [ ]:
%%capture
!pip install sentencepiece
!pip install transformers 

In [5]:
!wget https://raw.githubusercontent.com/shitkov/news_classification/main/train_data.csv

--2021-08-31 08:09:55--  https://raw.githubusercontent.com/shitkov/news_classification/main/train_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 417763 (408K) [text/plain]
Saving to: ‘train_data.csv.2’

train_data.csv.2    100%[===================>] 407.97K  --.-KB/s    in 0.02s   

2021-08-31 08:09:55 (18.1 MB/s) - ‘train_data.csv.2’ saved [417763/417763]



In [2]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-base-multitask")
model = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-base-multitask")

def generate(text, **kwargs):
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        hypotheses = model.generate(**inputs, num_beams=5, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

In [6]:
import pandas as pd
data = pd.read_csv('/content/train_data.csv', index_col=0)

In [7]:
from tqdm import tqdm

In [9]:
prefix = 'headline | '
texts = list(data['sentence'])
titles = []
for text in tqdm(texts):
    titles.append(generate(prefix + text))

100%|██████████| 1669/1669 [1:01:29<00:00,  2.21s/it]


In [10]:
data['title'] = titles

In [11]:
data.to_csv('titles.csv')

In [ ]:
# augmentation: paraphrase